```javascript
const contactData = {
    phone: "(+45) 6165 3935",
    emails: ["ole@covalente.dk", "ovalente@grundfos.com"],
    mastodon: "https://social.vivaldi.net/@ovalente",
    threema: "https://threema.id/A3ABK2PF"
};
```
Setup Encryption Functions

Write a helper function to convert ArrayBuffer to Base64:
```javascript
function arrayBufferToBase64(buffer) {
    let binary = '';
    const bytes = new Uint8Array(buffer);
    bytes.forEach(b => binary += String.fromCharCode(b));
    return window.btoa(binary);
}
```


Encrypt the Data

Use the Web Crypto API to encrypt the JSON payload with the password "Password1!" using PBKDF2 (to derive an AES‑128 key) and AES‑CBC encryption. Here’s a complete function:
```javascript
async function encryptData(data, password) {
    const encoder = new TextEncoder();
    // Convert data object to JSON and then to a Uint8Array
    const dataStr = JSON.stringify(data);
    const dataBytes = encoder.encode(dataStr);

    // Generate a random salt (16 bytes) and initialization vector (iv) (16 bytes)
    const salt = window.crypto.getRandomValues(new Uint8Array(16));
    const iv = window.crypto.getRandomValues(new Uint8Array(16));

    // Import the password material for PBKDF2 key derivation
    const keyMaterial = await window.crypto.subtle.importKey(
        "raw",
        encoder.encode(password),
        "PBKDF2",
        false,
        ["deriveKey"]
    );

    // Derive a 128-bit AES key using PBKDF2
    const key = await window.crypto.subtle.deriveKey(
        {
            name: "PBKDF2",
            salt: salt,
            iterations: 100000,
            hash: "SHA-256"
        },
        keyMaterial,
        { name: "AES-CBC", length: 128 },
        false,
        ["encrypt"]
    );

    // Encrypt the data using AES-CBC
    const ciphertextBuffer = await window.crypto.subtle.encrypt(
        {
            name: "AES-CBC",
            iv: iv
        },
        key,
        dataBytes
    );

    // Convert salt, iv, and ciphertext to Base64 strings
    return {
        salt: arrayBufferToBase64(salt.buffer),
        iv: arrayBufferToBase64(iv.buffer),
        ciphertext: arrayBufferToBase64(ciphertextBuffer)
    };
}
```

Run the Encryption and Log the Result

Finally, call the function with your contact data and password:
```javascript
encryptData(contactData, "Password1!")
    .then(encryptedPayload => {
        console.log("Encrypted JSON payload:", JSON.stringify(encryptedPayload, null, 2));
        // You can now copy the JSON and embed it into your HTML like so:
        // <div id="contact-encrypted-data" data-oval='[YOUR_ENCRYPTED_JSON_HERE]'></div>
    })
    .catch(error => {
        console.error("Encryption error:", error);
    });
```

In [2]:
import os
import json
import base64
import argparse
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def encrypt_data(data, password):
    # If data is not a string, convert (e.g., list or dict) to JSON string
    if not isinstance(data, str):
        data = json.dumps(data)
    data_bytes = data.encode('utf-8')

    # Generate random salt (16 bytes) and iv (16 bytes)
    salt = os.urandom(16)
    iv = os.urandom(16)

    # Derive a 128-bit key using PBKDF2 and SHA-256
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=16,  # for AES-128
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode('utf-8'))

    # Pad the data for AES blocksize
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(data_bytes) + padder.finalize()

    # Encrypt using AES-CBC
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    # Return Base64 encoded parts
    return {
        "salt": base64.b64encode(salt).decode('utf-8'),
        "iv": base64.b64encode(iv).decode('utf-8'),
        "ciphertext": base64.b64encode(ciphertext).decode('utf-8')
    }


In [ ]:

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Encrypt data for web encryption/decryption")
    parser.add_argument('--input', type=str, required=True,
                        help="Input string or file path")
    parser.add_argument('--password', type=str, default="Password1!",
                        help="Password to derive the encryption key (default: Password1!)")
    parser.add_argument('--is_file', action='store_true',
                        help="Set if the input argument is a file path")
    args = parser.parse_args()

    if args.is_file:
        # Read the file content
        with open(args.input, "r", encoding='utf-8') as f:
            data = f.read()
    else:
        # Use the input string/array (if array, should be passed as JSON string)
        data = args.input

    encrypted_payload = encrypt_data(data, args.password)
    # Print out JSON; this can be embedded via something like data-xxxxx attribute
    print("Encrypted data:", json.dumps(encrypted_payload, indent=2))

In [2]:

with open("./private.vcf", "r", encoding='utf-8') as f:
    data = f.read()

    encrypted_payload = encrypt_data(data, "Password1!")
    # Print out JSON; this can be embedded via something like data-xxxxx attribute
    print("Encrypted data:", json.dumps(encrypted_payload, indent=2))

Encrypted data: {
  "salt": "NaJCb9H52zrn7s1ULWMZYw==",
  "iv": "SgCxs8YgQEDAZQ9Exk6AGA==",
  "ciphertext": "..."
}
